***

* [Outline](../0_Introduction/0_introduction.ipynb)
* [Glossary](../0_Introduction/1_glossary.ipynb)
* [6. Deconvolution in Imaging](6_0_introduction.ipynb)  
    * Previous: [6.2 Point Source Assumption (CLEAN)](6_2_clean.ipynb)  
    * Next: [6.4 Source Finding and Detection](#)

***

Import standard modules:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Import section specific modules:

In [ ]:
import matplotlib.image as mpimg

***

## 6.3 Residuals and Image Quality<a id='deconv:sec:iqa'></a>

**TODO**

1. problem with clean: knowing when to stop, what makes a good image, how do you judge the quality of an image?
2. standard quality measurement (SNR), advantage is that it is easy to compute, disadvantage: says nothing about localized structure, example: show two images with the same SNR but one obviously better quality
3. residuals: what remains, what is the cause of the remaining structure (over/under clean, w-term, large scale structure of extended sources, poor clean settings)
4. image quality assessment is really hard to automate because it is a complex opinion which is based on previous experience and is hard to reduce down to simple metrics, this needs more work
5. once we are happy with the sky model and restored image, what now? we need to make a judgement call about what is a source, and what is noise, and if sky model components are localized enough to be considered part of the same source. for this we need source finding.

***

Next: [6.4 Source Finding and Detection](#)